*Aluna:* Maria Clara Miguel Claudino  
*Disciplina*: Métodos Numéricos

### Decomposição LU

1) Mostre o passo a passo da obtenção da matriz triangular inferior no processo de decomposição LU para a matriz mostrada no quadro.

A Decomposição LU é um método para quebrar uma matriz $A$ em duas: uma matriz $L$ (triangular inferior) e uma matriz $U$ (triangular superior), de forma que:  
$A = LU$.

Primeiro, fazemos a eliminação Gaussiana na matriz $A$ pra transformá-la na matriz $U$.  
Os "multiplicadores" que usamos para zerar os elementos abaixo da diagonal (os $l_{ij}$) são guardados e formam a matriz $L$.  
A matriz $L$ tem 1s na diagonal principal.  

Tendo $L$ e $U$, resolver $Ax=b$ vira duas etapas:  
- Primeiro, resolve $Ly = b$ (facil, porque $L$ é triangular inferior - substituição pra frente).  
    - Depois, resolve $Ux = y$ (facil, porque $U$ é triangular superior - substituição pra trás).  

O $x$ que sai no final é a solução do sistema.

2) Construa uma função em Python que faça a decomposição LU de uma dada matriz.

In [2]:
import numpy as np

def resolver_sistema_linear_lu(matriz_a, vetor_b):

    dimensao = matriz_a.shape[0]

    if matriz_a.shape[1] != dimensao:
        print("Erro: A matriz deve ser quadrada.")
        return None
    if vetor_b.shape[0] != dimensao:
        print("Erro: O tamanho do vetor b deve ser igual à dimensão da matriz.")
        return None

    matriz_l = np.eye(dimensao, dtype=float)
    matriz_u = np.copy(matriz_a).astype(float)

    for coluna_pivo in range(dimensao):
        if matriz_u[coluna_pivo, coluna_pivo] == 0:
            print(f"Aviso: Elemento pivô U[{coluna_pivo},{coluna_pivo}] é zero. "
                  "Decomposição LU pode não ser possível sem pivoteamento.")
            return None
        for linha_atual in range(coluna_pivo + 1, dimensao):
            multiplicador = matriz_u[linha_atual, coluna_pivo] / matriz_u[coluna_pivo, coluna_pivo]
            matriz_l[linha_atual, coluna_pivo] = multiplicador
            matriz_u[linha_atual, coluna_pivo:] -= multiplicador * matriz_u[coluna_pivo, coluna_pivo:]
    
    print("Matriz Triangular Inferior L:")
    print(matriz_l)
    print("\nMatriz Triangular Superior U:")
    print(matriz_u)

    vetor_y = np.zeros(dimensao, dtype=float)
    for i in range(dimensao):
        vetor_y[i] = (vetor_b[i] - np.dot(matriz_l[i, :i], vetor_y[:i])) / matriz_l[i, i]

    vetor_x = np.zeros(dimensao, dtype=float)
    for i in range(dimensao - 1, -1, -1):
        if matriz_u[i, i] == 0:
            print(f"Aviso: Elemento pivô U[{i},{i}] é zero durante a substituição regressiva. "
                  "Sistema pode ter múltiplas soluções ou nenhuma.")
            return None
        vetor_x[i] = (vetor_y[i] - np.dot(matriz_u[i, i+1:], vetor_x[i+1:])) / matriz_u[i, i]
    
    print("\nSolução Final")
    print("Vetor Solução x:")
    print(vetor_x)
    print("\nVerificação A @ x (deve ser aproximadamente b):")
    print(matriz_a @ vetor_x)

    return vetor_x

In [3]:
A = np.array([
    [4, -1, 3, 8],
    [1, 6, 2, -3],
    [5, 5, 1, 0],
    [2, 4, -2, 1]
])

b = np.array([43, 7, 8, 8])

solucao = resolver_sistema_linear_lu(A, b)


Matriz Triangular Inferior L:
[[1.   0.   0.   0.  ]
 [0.25 1.   0.   0.  ]
 [1.25 1.   1.   0.  ]
 [0.5  0.72 1.1  1.  ]]

Matriz Triangular Superior U:
[[ 4.   -1.    3.    8.  ]
 [ 0.    6.25  1.25 -5.  ]
 [ 0.    0.   -4.   -5.  ]
 [ 0.    0.    0.    6.1 ]]

Solução Final
Vetor Solução x:
[-2.44262295  3.39344262  3.24590164  5.80327869]

Verificação A @ x (deve ser aproximadamente b):
[43.  7.  8.  8.]
